## <font color = yellow> A single Sale will combine the first row from the Buys & Sells dfs

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


## <font color = yellow> create_df function () - reads .csv

In [3]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [4]:
head(df, 2)


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


## <font color = yellow> Create dfs

In [5]:
df_records <- data.frame()
sale_records <- data.frame() 


In [6]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
b <- buys[1,]
s <- sells[1,]

## <font color = yellow> Basic updates when a sale occurs


            sale <- s$quan
            bs_ratio <- b$quan / s$quan
            cf_quan <-  b$quan - s$quan

            # assign the cf values - if they exist
            if(b$quan > s$quan) {
                b$cf_quan <- (b$quan - s$quan)        
                b$cf_pps <- b$pps
                b$cf_basis <- cf_quan * b$pps
            }
            b$sale_quan <- b$quan
            b$buy_quan <- s$quan 
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            b$sale <- sale

            s$type <- "SOLD"

            return(b)   



In [7]:
# stopps

## <font color = yellow> The "working" components needed to accomplish the tasks

In [8]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


In [9]:
# update b

update_b <- function() {

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
        #  b > s
        if ( b$quan > s$quan) {    
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"
    
       # ( s > b ), has sell carryforward
 # cat("If s > b, then S_cf_quan is: ", S_cf_quan, " and Sell_cf is ", b$Sell_cf, "\n")  
        }else if(s$quan > b$quan) {
            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            

            # b$sell_quan <- s$quan    # cannot sell more than the buy_quan
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            
            
            
            
            # b$sale <- sale

            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {
            
            
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "SOLD"
            
            }
            
    
    

        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


In [10]:
# updated_b <- update_b()
# updated_b

### <font color = lime> Works with each sale type:( b > s , s > b, b = s, buys_cf, sells_cf )

In [11]:
evaluate_sale_type <- function () {
    
    cat(" This function displays the Sale Type based on the quan values from the updated_b data. ", "\n")
    cat(" Since the updated_b file is being used for evalutions, KEEP THINGS SIMPLE.  Don't update buys / sells ", "\n\n")

    if(updated_b$quan > updated_b$sell_quan && updated_b$cf_quan > 0 ) {
        cat("This is a BUY carryforward transaction:  ( b_quan > s_quan ) ", "\n")
        cat("Since this is a BUY carryforward transaction, the carryforward values need to be appended to buys[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
    } else if(updated_b$quan == updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a simple transaction:  (b_quan == s_quan) ", "\n")
        cat(" T/F good to go.  Run snip buys[1,], sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop ", "\n\n")            
    }else if (updated_b$quan < updated_b$sell_quan && updated_b$cf_quan == 0 ) {
        cat("This is a SELL carryforward transaction:  ( b_quan < s_quan ) ", "\n")
        cat("Since this is SELL carryforward transaction, the carryforward values need to be appended to sells[1, ] position", "\n")
        cat(" T/F, good to go. Run:  snip buys[1, ] and sells[1,] ", "\n")
        cat(" And good to go. Record the updated_b into the permanent sales_record df ", "\n")
        cat(" And then begin a new loop " , "\n\n")
        }
        
        cat("May need to examime more Sale Types ... when there is / is not CF values in Buys / Sells from previous transaction ", "\n\n")
    
    }  # fct

evaluate_sale_type()

 This function displays the Sale Type based on the quan values from the updated_b data.  
 Since the updated_b file is being used for evalutions, KEEP THINGS SIMPLE.  Don't update buys / sells  

This is a BUY carryforward transaction:  ( b_quan > s_quan )  
Since this is a BUY carryforward transaction, the carryforward values need to be appended to buys[1, ] position 
 T/F, good to go. Run:  snip buys[1, ] and sells[1,]  
 And good to go. Record the updated_b into the permanent sales_record df  
 And then begin a new loop  

May need to examime more Sale Types ... when there is / is not CF values in Buys / Sells from previous transaction  



## <font color = yellow> Delete one row from buys and sells

In [12]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

buys <- buys[-1, ]
save_sell_one <- sells [1, ]
sells <- sells[-1, ]
nrow(buys)

[1] 23

[1] 22

## <font color = yellow> Update the sales_records df w/ updated_b

In [13]:
sale_records <- rbind(sale_records, updated_b)
sale_records
                      

index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,50,⋯,4.07,4070,1000,5.5124,5512.4,1442.4,0,0,0,0


## <font color = yellow> Bind  buys & sells - when buy/sell c/f exists


In [14]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


In [15]:
if (sale_records$cf_quan[1] == 0 ) {
    cat("There is NO Buy c/f quan ", sale_records$cf_quan, "\n")
    }

In [16]:
# Here is is nec to update Buys[1, ]
if (sale_records$cf_quan[1] > 0 ) {
    cat("There IS Buy c/f quan ", sale_records$cf_quan, "\n")
    temp <- sale_records[1,]
    temp$type <- "Buy CF"
    temp$quan <- temp$cf_quan
    temp$pps <- temp$cf_pps
    temp$basis <- temp$cf_basis
    
    temp[ , 8:ncol(temp)] <- 0
    test <- buys
    test <- test %>% add_row(temp, .before = 1)
    buys <- test
    }

There IS Buy c/f quan  50 


In [17]:
if (sale_records$Sell_cf[1] == 0 ) {
    cat("There is NO Sell c/f ", sale_records$Sell_quan, "\n")
    }

There is NO Sell c/f  


In [18]:
# Here it is nec to update the Sells [1, ] 
if (sale_records$Sell_cf[1] > 0 ) {
    cat("There IS Sell c/f ", sale_records$Sell_cf, "\n")
    
    temp <- save_sell_one
    temp2 <-  temp <- sale_records[1,]
    temp[ , 8:ncol(save_sell_one)] <- 0
   
    
    temp$type <- "Sell CF"
    temp$quan <- temp2$Sell_cf
    temp$pps <- temp2$Sell_cf_pps
    temp$basis <- temp2$Sell_cf_basis

    sell_one <- temp
    
    test <- sells
    test <- test %>% add_row(sell_one, .before = 1)
    sells <- test
    
    
    
    
    
    }

In [19]:
#  Here, there are no c/f issues , t/f , return()
if (sale_records$cf_quan[1] == 0 && sale_records$Sell_cf[1] == 0 ) {
    cat("There is NO Buy c/f or Sell c/f ", sale_records$cf_quan, " \t", sale_records$Sell_cf, "\n")
    }


In [21]:
nrow(buys)
head(buys,3)

[1] 23

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,2022-12-31,Buy CF,AMC,50,4.070,203.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,5,2023-01-23,Buy,AMC,1000,5.788,5788.0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [22]:
nrow(sells)
head(sells, 3)

[1] 17

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,6,2023-02-03,Sell,AMC,1000,6.25500,6255.00,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
8,8,2023-02-03,Sell,AMC,1000,6.19010,6190.10,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [ ]:
#  # Only once per loop
# test <- test %>% add_row(temp, .before = 1)
# head(test, 3)

### <font color = lime> Got it! 

In [ ]:
stopps #  Old code

## <font color = yellow> sell_snip & snips functions () - the snips will parse row 1, then call test_for_cf fct

In [ ]:
#  These functions will delete the first row from the Buys and Sells dfs 
#  concluding the first Sale event

i <- 1


# fct
snips <- function (b, buys, df_records) {
    
    b <- b
    buys <- buys  
    df_records <- df_records
    
    # remove the first row of buys ... is b 
    buys <- buys[-1, ]   
    
    
    # Only if, then call fct that will apend df_records for the c/f values.  Otherwise ... ? 
    if(df_records$cf_quan > 0 ) {
        
        #fct call
        buys <- test_for_cf(b, buys, df_records)        
        return(buys)
        
        }else if( df_records$cf_quan == 0 ) {  
                cat("else if ( df_records$cf_quan == 0 .... ??? ", "\n")
            return(buys)       
         }
    
    #  return(buys)
    
   }  # fct

In [ ]:
# fct
sell_snip <- function(sells) {
    sells <- sells    
    sells <- sells[-1, ]    
    return(sells)
    }


# fct call


# nrow(buys)
# nrow(sells)

# buys <- snips(buys)
# sells <- sell_snip(sells)
# nrow(buys)
# nrow(sells)
# head(buys, 3)

In [ ]:
df_records

In [ ]:
# These are the fcts (in order) to process a Sals, from start to end 

nrow(buys)
nrow(sells)

buys <- snips(b,buys, df_records)
sells <- sell_snip(sells)
nrow(buys)
nrow(sells)
head(buys, 3)

In [ ]:
# fct call: to create vars from the first row of Buy & Sell
b <- row_b(b)   
s <- row_s(s)  
b
s

In [ ]:
df_records

head(buys, 3)
head(sells,3)

In [ ]:
 stopps

# <font color = lime> Loop 1 works when there is a carryforward!

## <font color = yellow> Fresh Start: sequence of functions to process one FiFO Sale (one Buy/ one Sell) 
    
        dfs <- create_dfs() 
        row_b <- function(b) 
        row_s <- function(s) 
        when_cf  when_cf(b, s, df_buy_rows)
        df_records(df_records, b, i) 
        test_for_cf (b,buys, df_records)
        snips <- function (buys) 
        sell_snip <- function(sells) 


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells


In [ ]:
b <- row_b(b)   
s <- row_s(s)   

In [ ]:
df_records <- df_create_records(df_records, b, i)

In [ ]:
df_records <- when_cf(b, s, df_records)

In [ ]:
buys <- snips(b,buys, df_records)
sells <- sell_snip(sells)


In [ ]:
head(df_records)
head(buys,2)
head(sells,2)

In [ ]:
stopps

### <font color = yellow> works! Now combine in a loop

In [ ]:
row_b <- function(b) {
        b <- b
        b <- buys[i,]
        return(b)
    }

row_s <- function(s) {
        s <- s
        s <- sells[i,]
        return(s)
    }
  

# nrow(b)
# nrow(s)


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells

b <- row_b(b)   
s <- row_s(s)  

In [ ]:
for(i in 1:nrow(df[1:1, ])) {
    
    cat("test print ", i, "\n")
    
    b <- row_b(b)   
    s <- row_s(s)  
    
    df_records <- df_create_records(df_records, b, i)
    df_records <- when_cf(b, s, df_records)
    buys <- snips(b,buys, df_records)
    sells <- sell_snip(sells)
    
    }
    

In [ ]:
b
s

In [ ]:
head(df_records)
head(buys,3)
head(sells,3)

In [ ]:
stopps # end of loop 1

## <font color = yellow> Works as expected w/ one Buy - one Sell - one loop
    
#### <font color = yellow> See what happens using the next Buy / Sell - loop 2 (involves the c/f from Sale 1 )
    


    
    

In [ ]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 200)

In [ ]:
row_b <- function(b) {
        b <- b
        b <- buys[i,]
        return(b)
    }

row_s <- function(s) {
        s <- s
        s <- sells[i,]
        return(s)
    }
  

# nrow(b)
# nrow(s)


In [ ]:
b
s


In [ ]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
# df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells

b <- row_b(b)   
s <- row_s(s) 
df_records <- b

In [ ]:
b
s

In [ ]:
for(i in 1:nrow(df[1:2, ])) {
    
    cat("test print ", i, "\n")
    
    b <- buys[i,]
    s <- row_s(s)  
    b
    s
    # df_records <- df_create_records(df_records, b, i)
    df_records <- when_cf(b, s, df_records)
    buys <- snips(b,buys, df_records)
    sells <- sell_snip(sells)
    
    df_records[i,]
    # print(buys[i, ])
    # print(sells[i, ])
    cat("newline ", "\n\n")
    }
    

In [ ]:
head(df_records)

cat("--------------------------------------------------------", "\\n\n\n")
cat("--------------------------------------------------------", "\\n\n\n")
head(buys,3)
head(sells,3)